# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

I. J. M. Crossfield  ->  I. J. M. Crossfield  |  ['I. J. M. Crossfield']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']


Y. Wang  ->  Y. Wang  |  ['Y. Wang']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
Arxiv has 81 new papers today
          5 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/5 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2510.23709
extracting tarball to tmp_2510.23709... done.
Retrieving document from  https://arxiv.org/e-print/2510.23732


extracting tarball to tmp_2510.23732... done.
Retrieving document from  https://arxiv.org/e-print/2510.24087


/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found documentclass in tmp_2510.23732/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'photometry' from 'tmp_2510.23732/photometry.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'candidates.tex' from 'tmp_2510.23732/candidates.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


extracting tarball to tmp_2510.24087...

 done.
Retrieving document from  https://arxiv.org/e-print/2510.24395
extracting tarball to tmp_2510.24395...

 done.
Retrieving document from  https://arxiv.org/e-print/2510.24708


extracting tarball to tmp_2510.24708...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2510.24708-b31b1b.svg)](https://arxiv.org/abs/2510.24708) | **Horizontal and vertical exoplanet thermal structure from a JWST spectroscopic eclipse map**  |
|| R. C. Challener, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2025-10-29*|
|*Comments*| *Published in Nature Astronomy. First two authors contributed equally*|
|**Abstract**|            Highly-irradiated giant exoplanets known as "ultra-hot Jupiters" are anticipated to exhibit large variations of atmospheric temperature and chemistry as a function of longitude, latitude, and altitude. Previous observations have hinted at these variations, but the existing data have been fundamentally restricted to probing hemisphere-integrated spectra, thereby providing only coarse information on atmospheric gradients. Here we present a spectroscopic eclipse map of an extrasolar planet, resolving the atmosphere in multiple dimensions simultaneously. We analyze a secondary eclipse of the ultra-hot Jupiter WASP-18b observed with the NIRISS instrument on JWST. The mapping reveals weaker longitudinal temperature gradients than were predicted by theoretical models, indicating the importance of hydrogen dissociation and/or nightside clouds in shaping global thermal emission. Additionally, we identify two thermally distinct regions of the planet's atmosphere: a "hotspot" surrounding the substellar point and a "ring" near the dayside limbs. The hotspot region shows a strongly inverted thermal structure due to the presence of optical absorbers and a water abundance marginally lower than the hemispheric average, in accordance with theoretical predictions. The ring region shows colder temperatures and poorly constrained chemical abundances. Similar future analyses will reveal three-dimensional thermal, chemical, and dynamical properties of a broad range of exoplanet atmospheres.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2510.23709-b31b1b.svg)](https://arxiv.org/abs/2510.23709) | **Sensitivity to Sub-Io-sized Exosatellite Transits in the MIRI LRS Lightcurve of the Nearest Substellar Worlds**  |
|| A. Householder, et al. -- incl., <mark>I. J. M. Crossfield</mark> |
|*Appeared on*| *2025-10-29*|
|*Comments*| *Published in ApJ Letters*|
|**Abstract**|            JWST's unprecedented sensitivity enables precise spectrophotometric monitoring of substellar worlds, revealing atmospheric variability driven by mechanisms operating across different pressure levels. This same precision now permits exceptionally sensitive searches for transiting exosatellites, small terrestrial companions to these worlds. Using a novel simultaneous dual-band search method to address host variability, we present a search for transiting exosatellites in an 8-hour JWST/MIRI LRS lightcurve of the nearby ($2.0\,pc$) substellar binary WISE J1049-5319AB, composed of two $\sim30 M_{\rm Jup}$ brown dwarfs separated by $3.5\,au$ and viewed near edge-on. Although we detect no statistically significant transits, our injection-recovery tests demonstrate sensitivity to satellites as small as $0.275\,R_{\oplus}$ ($0.96\,R_{\rm Io}$ or $\sim$1 lunar radius), corresponding to 300ppm transit depths, and satellite-to-host mass ratios $>$$10^{-6}$. This approach paves the way for detecting Galilean-moon analogs around directly imaged brown dwarfs, free-floating planets, and wide-orbit exoplanets, dozens of which are already scheduled for JWST lightcurve monitoring. In our Solar System, each giant planet hosts on average 3.5 moons above this threshold, suggesting that JWST now probes a regime where such companions are expected to be abundant. The technique and sensitivities demonstrated here mark a critical step toward detecting exosatellites and ultimately enabling constraints on the occurrence rates of small terrestrial worlds orbiting $1\text{-}70$$M_{\rm Jup}$ hosts.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2510.23732-b31b1b.svg)](https://arxiv.org/abs/2510.23732) | **ZTF25abjmnps (AT2025ulz) and S250818k: A Candidate Superkilonova from a Sub-threshold Sub-Solar Gravitational Wave Trigger**  |
|| M. M. Kasliwal, et al. -- incl., <mark>Y. Wang</mark> |
|*Appeared on*| *2025-10-29*|
|*Comments*| *Revised for ApJL (submitted on 10/2/25)*|
|**Abstract**|            On August 18, 2025, the LIGO-Virgo-KAGRA collaboration reported gravitational waves from a sub-threshold binary neutron star merger. If astrophysical, this event would have a surprisingly low chirp mass, suggesting that at least one neutron star was below a solar mass. The Zwicky Transient Facility mapped the coarse localization and discovered a transient, ZTF25abjmnps (AT2025ulz), that was spatially and temporally coincident with the gravitational wave trigger. The first week of follow-up suggested properties reminiscent of a GW170817-like kilonova. Subsequent follow-up suggests properties most similar to a young, stripped-envelope, Type IIb supernova. Although we cannot statistically rule out chance coincidence, we undertake due diligence analysis to explore the possible association between ZTF25abjmnps and S250818k. Theoretical models have been proposed wherein sub-solar neutron star(s) may form (and subsequently merge) via accretion disk fragmentation or core fission inside a core-collapse supernova i.e. a ``superkilonova". Here, we qualitatively discuss our multi-wavelength dataset in the context of the superkilonova picture. Future higher significance gravitational wave detections of sub-solar neutron star mergers with extensive electromagnetic follow-up would conclusively resolve this tantalizing multi-messenger association.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2510.24087-b31b1b.svg)](https://arxiv.org/abs/2510.24087) | **CSST Slitless Spectra: Target Detection and Classification with YOLO**  |
|| Y. Zhou, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2025-10-29*|
|*Comments*| *30 pages, 12+6 figures. For high-resolution figures, please see the published version*|
|**Abstract**|            Addressing the spatial uncertainty and spectral blending challenges in CSST slitless spectroscopy, we present a deep learning-driven, end-to-end framework based on the You Only Look Once (YOLO) models. This approach directly detects, classifies, and analyzes spectral traces from raw 2D images, bypassing traditional, error-accumulating pipelines. YOLOv5 effectively detects both compact zero-order and extended first-order traces even in highly crowded fields. Building on this, YOLO11 integrates source classification (star/galaxy) and discrete astrophysical parameter estimation (e.g., redshift bins), showcasing complete spectral trace analysis without other manual preprocessing. Our framework processes large images rapidly, learning spectral-spatial features holistically to minimize errors. We achieve high trace detection precision (YOLOv5) and demonstrate successful quasar identification and binned redshift estimation (YOLO11). This study establishes machine learning as a paradigm shift in slitless spectroscopy, unifying detection, classification, and preliminary parameter estimation in a scalable system. Future research will concentrate on direct, continuous prediction of astrophysical parameters from raw spectral traces.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2510.24395-b31b1b.svg)](https://arxiv.org/abs/2510.24395) | **Optimising Underwater Neutrino Telescopes for All-Flavour Point Source Sensitivity**  |
|| I. Morton-Blake, et al. -- incl., <mark>Y. Wang</mark> |
|*Appeared on*| *2025-10-29*|
|*Comments*| *16 pages, 7 figures*|
|**Abstract**|            High-energy neutrino astronomy has advanced rapidly in recent years, with IceCube, KM3NeT, and Baikal-GVD establishing a diffuse astrophysical flux and pointing to promising source candidates. These achievements mark the transition from first detections to detailed source studies, motivating next-generation detectors with larger volumes, improved angular resolution, and full neutrino-flavour sensitivity. We present a performance study of large underwater neutrino telescopes, taking the proposed TRIDENT array in the South China Sea as a case study, with a focus on comparing the performance of various detector configurations against the TRIDENT baseline design. Both track-like events primarily from muon neutrinos, which provide precise directional information, and cascade events from all flavours, which offer superior energy resolution, diffuse-source sensitivity, and all-sky flavour coverage, are included to achieve a balanced performance across source types. The time to discover potential astrophysical sources with both track- and cascade-like events is used as the figure of merit to compare a variety of detector design choices. Our results show that, for a fixed number of optical modules, simply enlarging the instrumented volume does not inherently lead to improved performance, while taller strings can provide modest gains across all detector channels, within engineering constraints. Distributing dense clusters of strings over a large volume is found to generally worsen discovery potential compared to the baseline layout. Finally, the optical properties of the sea-water emerge as the key factor dictating the optimisation of detector layout, highlighting the need for in-situ measurements and early deployment of optical modules to guide the final array configuration.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

110  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

3  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

0  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
